# Contextual Compression Retriever（上下文壓縮檢索器）

## 概覽

```ContextualCompressionRetriever``` 是 LangChain 中一個強大的工具，旨在透過根據上下文壓縮所檢索的文件，來優化檢索流程。當你需要處理大量數據並根據特定查詢進行摘要或動態過濾時，此工具特別有用，能確保僅傳遞最相關的資訊至後續處理階段。

### 核心特點：

- **上下文感知壓縮**：根據查詢或上下文對文件進行壓縮，避免冗餘並保留關鍵資訊。
- **靈活整合**：可與其他 LangChain 元件無縫整合，易於納入既有工作流程。
- **壓縮方式可自訂**：可選擇不同壓縮技術（如摘要模型、向量嵌入等），滿足特定任務需求。

### 適用場景：

- 在問答系統中摘要大量資料。
- 為聊天機器人提供更簡潔、相關的回覆。
- 提升處理法律分析或學術研究等文件密集任務的效率。

透過此檢索器，開發者可顯著降低計算負擔，並提升向使用者呈現資訊的品質。

![](./assets/02-contextual-compression-retriever-workflow.png)  

---

## 目錄

- [概覽](#概覽)
- [環境設置](#環境設置)
- [基本檢索器設定](#基本檢索器設定)
- [上下文壓縮](#上下文壓縮)
- [使用 LLM 進行文件過濾](#使用-llm-進行文件過濾)
- [建立壓縮器與文件轉換器管線](#建立壓縮器與文件轉換器管線)

---

## 參考資料

- [如何使用上下文壓縮進行檢索](https://python.langchain.com/docs/how_to/contextual_compression/)
- [LLM ChainFilter 文件過濾器](https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.document_compressors.chain_filter.LLMChainFilter.html)

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.


In [114]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [ ]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langchain",
        "langchain_openai",
        "langchain_community",
        "langchain_text_splitters",
        "langchain_core",
        "faiss-cpu",
    ],
    verbose=False,
    upgrade=False,
)

In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "Contextual Compression Retriever",
    }
)

Environment variables have been set successfully.


置換 ```OPENAI_API_KEY``` in ```.env``` 檔案在跟目錄。 

[Note] This is not necessary if you've already set ```OPENAI_API_KEY``` in previous steps.

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

#### 下面的函數“用來以具有視覺吸引力的格式呈現文件。”


In [3]:
# 輔助函式：以較美觀的格式輸出文件內容
def pretty_print_docs(docs):
    """
    輸入參數：
        docs: 文件列表（通常是 LangChain 檢索結果，每個元素可能是 Document 物件，
              其中包含 page_content 屬性存放文字內容）
    功能：
        將文件列表以「分隔線 + 文件標題 + 內容」的方式輸出，方便閱讀
    """
    print(
        # 使用 join 將多個文件字串用分隔線連接起來
        f"\n{'-' * 100}\n".join(
            [
                # 為每份文件加上「文件序號」標題，並附上文件的 page_content（內容）
                f"document {i+1}:\n\n" + d.page_content
                for i, d in enumerate(docs)  # enumerate 用於同時取得索引與文件
            ]
        )
    )

## 基本檢索器設定（Basic Retriever Configuration）

我們先從初始化一個簡單的向量資料庫檢索器（Vector Store Retriever）開始，並將文字文件分段儲存。

當使用者提出一個問題時，這個檢索器會返回 1 至 2 筆相關文件，同時也可能包含一些不相關的文件，這反映出沒有經過上下文壓縮的檢索結果可能會混入雜訊。

以下是我們建立檢索器的步驟：

1. **使用 `TextLoader` 載入文字檔案**  
   利用 LangChain 提供的 `TextLoader` 將 `.txt` 檔案中的文字載入至記憶體中。

2. **使用 `CharacterTextSplitter` 將文本分段**  
   將文字分成每段 300 個字元，且設定 `chunk_overlap=0`（不重疊），方便後續向量化處理。

3. **建立向量資料庫（使用 FAISS）並轉為 Retriever**  
   將文本段進行嵌入向量轉換並存入 FAISS 向量庫，最後使用 `.as_retriever()` 方法轉換成可查詢的檢索器。

4. **向檢索器發送查詢問題**  
   輸入一個自然語言問題，讓檢索器返回與問題最相關的內容段落。

5. **列印相關段落內容**  
   檢視檢索器返回的結果，以了解是否準確命中關鍵資訊。

> 備註：這個基本設定將作為後續「上下文壓縮檢索器」的基礎。

In [4]:
from langchain_community.document_loaders import TextLoader   # 用來載入純文字檔案的文件載入器
from langchain_community.vectorstores import FAISS            # 向量資料庫 FAISS（用於相似度檢索）
from langchain_openai import OpenAIEmbeddings                  # OpenAI 的文字嵌入模型
from langchain_text_splitters import CharacterTextSplitter     # 文本切割工具（按字元長度分割）

# 1. 使用 TextLoader 載入文字檔案
#    這裡會將 ./data/appendix-keywords.txt 檔案讀入為一個 Document 物件
loader = TextLoader("./data/appendix-keywords.txt")

# 2. 使用 CharacterTextSplitter 將文字檔分割成小區塊
#    chunk_size=300 表示每個文本區塊最多 300 個字元
#    chunk_overlap=0 表示區塊之間沒有重疊
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = loader.load_and_split(text_splitter)  # 載入並切割成多個文本區塊

# 3. 使用 FAISS 建立向量資料庫，並將其轉為檢索器（retriever）
#    - OpenAIEmbeddings() 會將文字轉換為向量表示
#    - from_documents() 建立向量索引
#    - as_retriever() 將其封裝成可以用 query 查詢的檢索器
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever()

# 4. 使用檢索器進行查詢
#    查詢內容是 "What is the definition of Multimodal?"
#    retriever.invoke() 會回傳與查詢最相關的文件列表
docs = retriever.invoke("What is the definition of Multimodal?")

# 5. 以美觀的方式輸出檢索結果
#    這裡假設已經定義了 pretty_print_docs() 函式，用來格式化並印出每份文件內容
pretty_print_docs(docs)

Created a chunk of size 380, which is longer than the specified 300
Created a chunk of size 343, which is longer than the specified 300
Created a chunk of size 304, which is longer than the specified 300
Created a chunk of size 341, which is longer than the specified 300
Created a chunk of size 349, which is longer than the specified 300
Created a chunk of size 330, which is longer than the specified 300
Created a chunk of size 385, which is longer than the specified 300
Created a chunk of size 349, which is longer than the specified 300
Created a chunk of size 413, which is longer than the specified 300
Created a chunk of size 310, which is longer than the specified 300
Created a chunk of size 391, which is longer than the specified 300
Created a chunk of size 330, which is longer than the specified 300
Created a chunk of size 325, which is longer than the specified 300
Created a chunk of size 349, which is longer than the specified 300
Created a chunk of size 321, which is longer tha

document 1:

Multimodal
Definition: Multimodal refers to the technology that combines multiple types of data modes (e.g., text, images, sound) to process and extract richer and more accurate information or predictions.
Example: A system that analyzes both images and descriptive text to perform more accurate image classification is an example of multimodal technology.
Relate
----------------------------------------------------------------------------------------------------
document 2:

Semantic Search
----------------------------------------------------------------------------------------------------
document 3:

LLM (Large Language Model)
----------------------------------------------------------------------------------------------------
document 4:

Embedding


In [10]:
print(docs[0].page_content)

Multimodal
Definition: Multimodal refers to the technology that combines multiple types of data modes (e.g., text, images, sound) to process and extract richer and more accurate information or predictions.
Example: A system that analyzes both images and descriptive text to perform more accurate image classification is an example of multimodal technology.
Relate


## 上下文壓縮（Contextual Compression）

使用 ```LLMChainExtractor``` 所建立的 ```DocumentCompressor``` 正是應用於檢索器（Retriever）之上的壓縮器，也就是我們要使用的 ```ContextualCompressionRetriever```。

```ContextualCompressionRetriever``` 的功能是根據查詢的上下文對檢索回來的文件進行壓縮，**刪除不相關資訊，保留最重要的內容**，以提升回答品質與效率。

我們將比較應用壓縮前與壓縮後的檢索結果，來了解上下文壓縮帶來的改變。

In [5]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import ChatOpenAI

# Before applying ContextualCompressionRetriever
pretty_print_docs(retriever.invoke("What is the definition of Multimodal?"))
print("="*62)
print("="*15 + "After applying LLMChainExtractor" + "="*15)


# After applying ContextualCompressionRetriever
# 1. Generate LLM
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")  

# 2. Generate compressor using LLMChainExtractor
compressor = LLMChainExtractor.from_llm(llm)

# 3. Generate compression retriever using ContextualCompressionRetriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever,
)

# 4. Query the compression retriever to find relevant documents
compressed_docs = (
    compression_retriever.invoke( 
        "What is the definition of Multimodal?"
    )
)

# 5. Print the relevant documents
pretty_print_docs(compressed_docs)

document 1:

Multimodal
Definition: Multimodal refers to the technology that combines multiple types of data modes (e.g., text, images, sound) to process and extract richer and more accurate information or predictions.
Example: A system that analyzes both images and descriptive text to perform more accurate image classification is an example of multimodal technology.
Relate
----------------------------------------------------------------------------------------------------
document 2:

Semantic Search
----------------------------------------------------------------------------------------------------
document 3:

LLM (Large Language Model)
----------------------------------------------------------------------------------------------------
document 4:

Embedding
===============After applying LLMChainExtractor===============
document 1:

Multimodal
Definition: Multimodal refers to the technology that combines multiple types of data modes (e.g., text, images, sound) to process and extract

## Document Filtering Using LLM


### LLMChainFilter

### 使用 LLM 進行文件篩選（LLMChainFilter）

```LLMChainFilter``` 是一種簡潔但強大的壓縮器（Compressor），它透過 LLM Chain 判斷哪些文件應該被保留，哪些應該被過濾掉，適用於初步檢索後的文件集合。

這個篩選器的特點是**不改變原始文件內容**，而是**選擇性地回傳符合查詢上下文的文件**，用於提升檢索的精準度與效能。

In [6]:
from langchain.retrievers.document_compressors import LLMChainFilter

# 1. Generate LLMChainFilter object using LLM
_filter = LLMChainFilter.from_llm(llm)

# 2. Generate ContextualCompressionRetriever object using LLMChainFilter and retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter,
    base_retriever=retriever,
)

# 3. Query the compression retriever to find relevant documents
compressed_docs = compression_retriever.invoke(
    "What is the definition of Multimodal?"
)

# 4. Print the relevant documents
pretty_print_docs(compressed_docs)  

document 1:

Multimodal
Definition: Multimodal refers to the technology that combines multiple types of data modes (e.g., text, images, sound) to process and extract richer and more accurate information or predictions.
Example: A system that analyzes both images and descriptive text to perform more accurate image classification is an example of multimodal technology.
Relate


### EmbeddingsFilter

### 使用 EmbeddingsFilter 節省成本與提升效率

對每一份檢索到的文件進行額外的 LLM 呼叫會導致成本高昂且推理速度變慢。

```EmbeddingsFilter``` 提供一種更經濟且快速的替代方案，它透過將**查詢與文件進行向量嵌入（embedding）**，僅回傳與查詢**相似度足夠高的文件**。

這種方法能在保持檢索結果相關性的同時，**大幅節省計算成本與時間**。

整個流程如下：

- 使用 ```ContextualCompressionRetriever``` 結合 ```EmbeddingsFilter``` 進行壓縮式檢索。
- ```EmbeddingsFilter``` 將過濾掉與查詢**相似度低於設定門檻（例如 0.86）**的文件，只保留高相關性的內容供後續處理。

這使得在不犧牲品質的情況下，加速了檢索流程，特別適用於資源有限或需即時回應的應用場景。

In [ ]:
from langchain.retrievers.document_compressors import EmbeddingsFilter  # 基於嵌入向量的文件過濾器
from langchain_openai import OpenAIEmbeddings  # OpenAI 的文字嵌入模型

# 1. 建立向量嵌入模型
#    - OpenAIEmbeddings 會將文字轉換成向量，用於計算相似度
embeddings = OpenAIEmbeddings()

# 2. 建立 EmbeddingsFilter 對象
#    - embeddings: 使用上面建立的 OpenAIEmbeddings 來計算文本與查詢的相似度
#    - similarity_threshold=0.86：設定相似度門檻（0~1之間）
#      只有相似度 >= 0.86 的文件段落才會被保留，其餘會被過濾掉
embeddings_filter = EmbeddingsFilter(
    embeddings=embeddings, 
    similarity_threshold=0.86
)

# 3. 建立 ContextualCompressionRetriever
#    - base_compressor 使用 EmbeddingsFilter（基於相似度過濾）
#    - base_retriever 使用原本的檢索器 retriever（從向量庫中找出候選文件）
#    作用流程：
#      a. base_retriever 找出所有可能相關的文件
#      b. base_compressor 計算這些文件與查詢的相似度
#      c. 保留高於 0.86 閾值的內容
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, 
    base_retriever=retriever
)

# 4. 使用壓縮檢索器查詢
#    - 查詢「What is the definition of Multimodal?」
#    - 只會得到和查詢在語意上高度相關的文件段落
compressed_docs = compression_retriever.invoke(
    "What is the definition of Multimodal?"
)

# 5. 以美觀格式輸出相關文件
pretty_print_docs(compressed_docs)

document 1:

Multimodal
Definition: Multimodal refers to the technology that combines multiple types of data modes (e.g., text, images, sound) to process and extract richer and more accurate information or predictions.
Example: A system that analyzes both images and descriptive text to perform more accurate image classification is an example of multimodal technology.
Relate


## Creating a Pipeline (Compressor + Document Converter)

### 使用 `DocumentCompressorPipeline` 組合多種壓縮器與轉換器

透過 ```DocumentCompressorPipeline```，可以**串接多個壓縮器（Compressor）與文件轉換器（Transformer）**，依序執行處理流程。

除了壓縮器，也可以加入 ```BaseDocumentTransformer```，針對文件集合執行非語境壓縮的轉換操作。例如：

- 可以使用 ```TextSplitter``` 將文件切分為較小的段落，以利後續檢索或摘要。
- 可以使用 ```EmbeddingsRedundantFilter``` 根據嵌入相似度來移除重複文件（預設設定為相似度 ≥ 0.95 視為重複）。

以下為壓縮流程範例：
1. **文件切分**：先使用 `TextSplitter` 將原始文件分段。
2. **移除重複**：利用 `EmbeddingsRedundantFilter` 去除語意重複的段落。
3. **語意過濾**：依照與查詢的相似度，篩選出最相關的文件。

最終結果會是一組高度相關、非重複、且已預處理的小型文件段，有助於提升檢索效率與語言模型回答品質。

In [15]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline  # 文件壓縮管線
from langchain_community.document_transformers import EmbeddingsRedundantFilter   # 向量冗餘過濾器
from langchain_text_splitters import CharacterTextSplitter                        # 文本切割工具

# 1. 建立文本切割器 (CharacterTextSplitter)
#    - chunk_size=300：每個文本塊最大 300 個字元
#    - chunk_overlap=0：不同塊之間沒有重疊
#    目的：將長文本拆成小塊，方便後續向量化與壓縮處理
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 2. 建立冗餘過濾器 (EmbeddingsRedundantFilter)
#    - embeddings：使用之前建立的 OpenAIEmbeddings
#    作用：檢測文本塊之間的語意相似度，過濾掉與其他塊幾乎重複的段落
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

# 3. 建立相關性過濾器 (EmbeddingsFilter)
#    - similarity_threshold=0.86：相似度需達到 0.86 才保留
#    作用：只保留與查詢在語意上高度相關的文本塊
relevant_filter = EmbeddingsFilter(
    embeddings=embeddings,
    similarity_threshold=0.86
)

# 4. 建立文件壓縮管線 (DocumentCompressorPipeline)
#    - transformers：依序執行的處理器列表
#      a. splitter：切割原始文件
#      b. redundant_filter：刪除語意重複的塊
#      c. relevant_filter：刪除與查詢無關的塊
#      d. LLMChainExtractor：用 LLM 從保留下來的文本中提取最重要的內容
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[
        splitter,
        redundant_filter,
        relevant_filter,
        LLMChainExtractor.from_llm(llm),  # 使用前面建立的 llm 物件
    ]
)

While initializing the  ```ContextualCompressionRetriever```, we use ```pipeline_compressor``` as the ```base_compressor``` and ```retriever``` as the ```base_retriever```.

In [16]:
# 5. Use pipeline_compressor as the base_compressor and retriever as the base_retriever to initialize ContextualCompressionRetriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor,
    base_retriever=retriever,
)

# 6. Query the compression retriever to find relevant documents
compressed_docs = compression_retriever.invoke(
    "What is the definition of Multimodal?"
)

# 7. Print the relevant documents
pretty_print_docs(compressed_docs)


document 1:

Multimodal refers to the technology that combines multiple types of data modes (e.g., text, images, sound) to process and extract richer and more accurate information or predictions.
